# SUSA CX Kaggle Capstone Project
## Part 4: Deep Learning in Keras and Submitting to Kaggle

### Table Of Contents
* [Introduction](#section1)
* [Initial Setup](#section2)
* [Deep Learning](#section3)
* [Final Kaggle Evaluation](#section4)   
* [Conclusion](#conclusion)
* [Additional Reading](#reading)


### Hosted by and maintained by the [Statistics Undergraduate Students Association (SUSA)](https://susa.berkeley.edu). Originally authored by [Patrick Chao](mailto:prc@berkeley.edu) & [Arun Ramamurthy](mailto:contact@arun.run).

<a id='section1'></a>
# SUSA CX Kaggle Capstone Project

Woohoo! You've made it to the end of the CX Kaggle Capstone Project! Congratulations on all of your hard work so far. We hope you've enjoyed this opportunity to learn new modeling techniques, some underlying mathematics, and even make new friends within CX. At this point, we've covered the entirety of the Data Science Workflow, linear regression, feature engineering, PCA, shrinkage, hyperparameter tuning, decision trees and even ensemble models. This week, we're going to finish off this whirlwind tour with a revisit to our old friend, Deep Learning. While the MNIST digit dataset was really interesting to look at as a cool toy example of the powers of DL, this time you're going to apply neural networks to your housing dataset for some hands-on practice using Keras. 

> ### CX Kaggle Competition & Final Kaggle Evaluation
After you get some practice with deep learning, this week we will be asking you and your team to select and finalize your best model, giving you the codespace to write up your finalized model and evaluate it by officially submitting your results to Kaggle. The winners of this friendly collab-etition will be honored at the SUSA Banquet next Friday, including prizes for the winning team! We also want to encourage and facilitate discussion between teams on why different models performed differently, and give you a chance to chat with other teams about their own experiences with the CX Kaggle Capstone. 

## Logistics

Most of the logistics are the same as last week, but we are repeating them here for your convenience. Please let us know if you or your teammates are feeling nervous about the pace of this project - remember that we are not grading you on your project, and we really try to make the notebooks relatively easy and fast to code through. If for any reason you are feeling overwhelmed or frustrated, please DM us or talk to us in person. We want all of you to have a productive, healthy, and fun time learning data science! If you have any suggestions or recommendations on how to improve, please do not hesitate to reach out!


### Mandatory Office Hours

Because this is such a large project, you and your team will surely have to work on it outside of meetings. In order to get you guys to seek help from this project, we are making it **mandatory** for you and your group to attend **two (2)** SUSA Office Hours over the next 4 weeks. This will allow questions to be answered outside of the regular meetings and will help promote collaboration with more experienced SUSA members.

The schedule of SUSA office hours are below:
https://susa.berkeley.edu/calendar#officehours-table

We understand that most of you will end up going to Arun or Patrick's office hours, but we highly encourage you to go to other people's office hours as well. There are many qualified SUSA mentors who can help and this could be an opportunity for you to meet them.

<a id='section2'></a>
# Initial Setup

To begin we will import all the necessary libraries and functions.

In [24]:
# Import statements
from sklearn import tree # There are lots of other models from this module you can try!
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression, Ridge
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.externals.six import StringIO  
from IPython.display import Image  

import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD,Adam
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
from keras import backend as K
sqrt=np.sqrt
import xgboost as xgb

We also define a few familiar functions that should be helpful to you later.

In [25]:
def get_features(data, col_list, y_name):
    """
    Function to return a numpy matrix of pandas dataframe features, given k column names and a single y column
    Outputs X, a n X k dimensional numpy matrix, and Y, an n X 1 dimensional numpy matrix.
    This is not a smart function - although it does drop rows with NA values. It might break. 
    
    data(DataFrame): e.g. train, clean
    col_list(list): list of columns to extract data from
    y_name(string): name of the column you to treat as the y column
    
    Ideally returns one np.array of shape (len(data), len(col_list)), and one of shape (len(data), len(col_list))
    """
    
    # keep track of numpy values
    feature_matrix = data[col_list + [y_name]].dropna().values
    np.random.shuffle(feature_matrix)
    return feature_matrix[:, :-1], feature_matrix[:, -1]

def get_loss(model, X,Y_true):
    """Returns square root of L2 loss (RMSE) from a model, X value input, and true y values
    
    model(Model object): model we use to predict values
    X: numpy matrix of x values
    Y_true: numpy matrix of true y values
    """
    Y_hat = model.predict(X)
    return get_RMSE(Y_hat,Y_true)

def get_RMSE(Y_hat,Y_true):
    """Returns square root of L2 loss (RMSE) between Y_hat and true values
    
    Y_true: numpy matrix of predicted y values
    Y_true: numpy matrix of true y values
    """
    return np.sqrt(np.mean((Y_true-Y_hat)**2))

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def get_train_and_val(X,Y):
    """Given the X and Y data, return the training and validation based on the split variable
    
    X: numpy matrix of x values
    Y: numpy matrix of y values
    split: value between 0 and 1 for the training split
    """
    
    Y = Y.reshape(Y.shape[0],)

    train_index,_ = get_train_val_indices(X,Y)

    y_train = Y.reshape(Y.shape[0],)
    y_train = Y[:train_index]
    x_train = X[:train_index,:]

    x_val = X[train_index:,:]
    y_val = Y[train_index:]
    return (x_train,y_train),(x_val,y_val)

def get_train_val_indices(X,Y=None,split=0.7):
    train_index = (int)(X.shape[0]*split)
    test_index =X.shape[0]-1
    return train_index,test_index

def select_columns_except(dframe, non_examples):
    """Returns all comlumns in dframe except those in non_examples."""
    all_cols = dframe.select_dtypes(include=[np.number]).columns.tolist()
    cond = lambda x: sum([x == col for col in non_examples]) >= 1
    return [x for x in all_cols if not cond(x)]

#Metric for keras
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))


## Data Loading

First, we need to load and clean the data. Although you may have cleaned data from `kaggle1`, we provide our solution for the cleaned housing data for your convenience. If you would like to view the completed data cleaning procedure, it has been updated in [kaggle1.ipynb](kaggle1.ipynb).

In [26]:
train = pd.read_csv('DATA/house-prices/train_cleaned.csv')
test = pd.read_csv('DATA/house-prices/test_cleaned.csv')
train = train.drop('Unnamed: 0',axis=1)
test = test.drop('Unnamed: 0',axis=1)
train.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,...,ExteriorWd Shng,ExteriorImStucc,ExteriorWdShing,ExteriorMetalSd,ExteriorCmentBd,ExteriorCemntBd,ExteriorPlywood,ExteriorBrkFace,ExteriorWd Sdng,ExteriorBrk Cmn
0,856,854,3,3,1,706.0,0.0,6,1,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1262,0,3,3,4,978.0,0.0,5,1,0.0,...,0,0,0,1,0,0,0,0,0,0
2,920,866,3,3,2,486.0,0.0,6,1,1.0,...,0,0,0,0,0,0,0,0,0,0
3,961,756,3,4,1,216.0,0.0,5,1,1.0,...,1,0,0,0,0,0,0,0,1,0
4,1145,1053,4,3,3,655.0,0.0,6,1,1.0,...,0,0,0,0,0,0,0,0,0,0


Same as before, we need to preprocess the data into `numpy` matrices and separate the `SalePrice` as the response variable.

In [27]:
feature_cols = select_columns_except(train, ['Id','SalePrice'])

X, Y = get_features(train, feature_cols, 'SalePrice')
(x_train,y_train),(x_val,y_val) = get_train_and_val(X,Y)

x_test = test.loc[:, test.columns != 'Id'].values
test_ids = test['Id'].values

We provide a function `model_prediction` that takes in a model and a set of features from the test set, and outputs the predictions into a vector. This should work with the `keras` neural networks, `sklearn` decision trees and random forests. 

In [28]:
def model_prediction(model,test=x_test):
    prediction = model.predict(test)
    return prediction.reshape(prediction.shape[0],)

## Model Example 1 : Random Forest
To help you get started, we supply a couple of naive models. Recall the three steps to modeling: model selection, training, and evaluation (validation or testing). Use the optimal parameters you found from grid search last week to tune the following random forest model.

In [29]:
####################
### MODEL DESIGN ###
####################
max_depth = 20
min_samples_leaf = 1
min_samples_split = 2
n_estimators = 100
model_rf = RandomForestRegressor(max_depth = max_depth,
                              min_samples_leaf = min_samples_leaf, min_samples_split = min_samples_split,
                              n_estimators = n_estimators, random_state = 0, bootstrap = True)

In [30]:
################
### TRAINING ###
################
model_rf = model_rf.fit(x_train, y_train)

In [31]:
def model_prediction(model,x_test=x_test):
    prediction = model.predict(x_test)
    return prediction.reshape(prediction.shape[0],)

In [32]:
##################
### EVALUATION ###
##################
loss = get_loss(model_rf, x_val,y_val)
print("Root Mean Squared Error loss on the Validation Set for our RF model: {:.2f}".format(loss))

Root Mean Squared Error loss on the Validation Set for our RF model: 30595.99


<a id='section3'></a>
# Deep Learning

From `kaggle3`, a strange tale:  
>We may imagine hyperparameters as a bunch of individual knobs we can turn to change our model. Consider that you are visiting your friend and staying at her place. However, you did not realize that she is actually an alien and her house is filled with very strange objects. When you head to bed, you attempt to use her shower, but see that her shower is has a dozen of knobs that control the temperature of the water coming out! We only have a single output to work off of, but many different knobs or *parameters* to adjust. If the water is too hot, we can turn random knobs until it becomes cold, and learn a bit about our environment. We may determine that some knobs are more or less sensitive, just like hyperparameters. Each knob in the shower is equivalent to a hyperparameter we can tune in a model.

## Model Example 2 : Neural Networks
Here is a very simple example of a neural network in Keras. It's performance is not fantastic to start, but mess around with tuning parameters and adding or subtracting layers and see what you can come up with!

In [33]:
####################
### MODEL DESIGN ###
####################
model_nn = Sequential()
model_nn.add(Dense(100, activation='relu', input_shape=(x_train.shape[1],)))
model_nn.add(Dense(50, activation='relu', input_shape=(x_train.shape[1],)))
model_nn.add(Dense(50, activation='relu', input_shape=(x_train.shape[1],)))
model_nn.add(Dense(25, activation='relu', input_shape=(x_train.shape[1],)))
model_nn.add(Dense(10, activation='relu', input_shape=(x_train.shape[1],)))
model_nn.add(Dense(1, activation='relu'))

model_nn.compile(optimizer=Adam(), loss = root_mean_squared_error, 
              metrics =[root_mean_squared_error])

In [34]:
################
### TRAINING ###
################
batch_size = 30
epochs = 200
learning_rate = 0.5
history = model_nn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))

Train on 1021 samples, validate on 439 samples
Epoch 1/200
1021/1021 [==============================] - 0s 448us/step - loss: 179125.7164 - root_mean_squared_error: 179125.7164 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 2/200
1021/1021 [==============================] - 0s 76us/step - loss: 179125.7159 - root_mean_squared_error: 179125.7159 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 3/200
1021/1021 [==============================] - 0s 87us/step - loss: 179125.7168 - root_mean_squared_error: 179125.7168 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 4/200
1021/1021 [==============================] - 0s 84us/step - loss: 179125.7170 - root_mean_squared_error: 179125.7170 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 5/200
1021/1021 [==============================] - 0s 78us/step - loss: 179125.7168 - root_mean_squared_error: 179125.7168 - val_loss: 185097.0019 - val_root_mean

1021/1021 [==============================] - 0s 84us/step - loss: 179125.7168 - root_mean_squared_error: 179125.7168 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 43/200
1021/1021 [==============================] - 0s 89us/step - loss: 179125.7161 - root_mean_squared_error: 179125.7161 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 44/200
1021/1021 [==============================] - 0s 88us/step - loss: 179125.7161 - root_mean_squared_error: 179125.7161 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 45/200
1021/1021 [==============================] - 0s 76us/step - loss: 179125.7167 - root_mean_squared_error: 179125.7167 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 46/200
1021/1021 [==============================] - 0s 81us/step - loss: 179125.7168 - root_mean_squared_error: 179125.7168 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 47/200
1021/1021 [====

Epoch 84/200
1021/1021 [==============================] - 0s 73us/step - loss: 179125.7162 - root_mean_squared_error: 179125.7162 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 85/200
1021/1021 [==============================] - 0s 75us/step - loss: 179125.7176 - root_mean_squared_error: 179125.7176 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 86/200
1021/1021 [==============================] - 0s 78us/step - loss: 179125.7180 - root_mean_squared_error: 179125.7180 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 87/200
1021/1021 [==============================] - 0s 70us/step - loss: 179125.7154 - root_mean_squared_error: 179125.7154 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 88/200
1021/1021 [==============================] - 0s 73us/step - loss: 179125.7179 - root_mean_squared_error: 179125.7179 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 89/200
10

1021/1021 [==============================] - 0s 67us/step - loss: 179125.7162 - root_mean_squared_error: 179125.7162 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 126/200
1021/1021 [==============================] - 0s 70us/step - loss: 179125.7176 - root_mean_squared_error: 179125.7176 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 127/200
1021/1021 [==============================] - 0s 76us/step - loss: 179125.7167 - root_mean_squared_error: 179125.7167 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 128/200
1021/1021 [==============================] - 0s 72us/step - loss: 179125.7167 - root_mean_squared_error: 179125.7167 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 129/200
1021/1021 [==============================] - 0s 75us/step - loss: 179125.7161 - root_mean_squared_error: 179125.7161 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 130/200
1021/1021 

1021/1021 [==============================] - 0s 69us/step - loss: 179125.7173 - root_mean_squared_error: 179125.7173 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 167/200
1021/1021 [==============================] - 0s 80us/step - loss: 179125.7170 - root_mean_squared_error: 179125.7170 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 168/200
1021/1021 [==============================] - 0s 73us/step - loss: 179125.7165 - root_mean_squared_error: 179125.7165 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 169/200
1021/1021 [==============================] - 0s 75us/step - loss: 179125.7165 - root_mean_squared_error: 179125.7165 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 170/200
1021/1021 [==============================] - 0s 71us/step - loss: 179125.7162 - root_mean_squared_error: 179125.7162 - val_loss: 185097.0019 - val_root_mean_squared_error: 185097.0019
Epoch 171/200
1021/1021 

In [35]:
##################
### EVALUATION ###
##################
score = model_nn.evaluate(x_val, y_val, verbose=0)
print("Root Mean Squared Error loss on the Validation Set for our neural net model: {:.2f}".format(score[0]))

Root Mean Squared Error loss on the Validation Set for our neural net model: 185097.00


<a id='section4'></a>
# Final Kaggle Evaluation

Congrats on finishing the last of the models we planned on teaching you about during the CX Kaggle Capstone! 

You have now covered five distinct models and a several related techniques to add to your data science bag-of-tricks: 
- Linear Models
    - Multivariate Linear Regression
    - Polynomial Regression
    - Shrinkage / Biased Regression / Regularization (i.e. Ridge, LASSO)
- Decision Trees
    - Random Forests
- Deep Learning
    - Sequential Neural Networks
- Auxiliary Techniques 
    - The Data Science Workflow
    - Data Cleaning
    - Interpreting EDA Graphs
    - Feature Engineering
    - Principal Component Analysis
    - Hyperparameter Tuning (i.e. grid search)
    - Ensemble Learning (i.e. bagging, boosting) 
    
Wow, that's a lot! We are really proud of you all for exploring these techniques, which constitute some of Berkeley's toughest machine learning and statistics classes. As always, if you want to learn more about any of these topics, or are hungry to learn about even more techniques, feel free to reach out to any one of the SUSA Mentors.

With the help of the above listing and your own team's preferences, choose a model and a couple of techniques to implement for your final model. We will provide you with a preamble and some space to construct and train your model, as well as a helper function to turn your output into an official Kaggle submission file.  

A huge part of the modeling process is to mess around with different models, approaches, and hyperparameters! Don't be afraid to get your hands dirty and explore!

In [112]:
# trying to do xgboost lol
boost = xgb.XGBRegressor(n_estimators=280, learning_rate=0.1, max_depth=3, colsample_bytree=1, reg_alpha=150,gamma=0.3, subsample=1)
boost.fit(x_train,y_train)
loss = get_loss(boost, x_val,y_val)
print(loss)

# for i in np.arange(1, 1000, 100):
#     avg_loss = []
#     for j in np.arange(5):
#         boost = xgb.XGBRegressor(n_estimators=i, learning_rate=0.05, gamma=0.3, subsample=1,
#                                colsample_bytree=1, max_depth=6, reg_alpha=150)
#         boost.fit(x_train, y_train)
#         loss = get_loss(boost, x_val, y_val)
#         avg_loss = np.append(avg_loss, loss)
#     print(np.mean(avg_loss))

24898.690418755177


In [109]:
################
### PREAMBLE ###
################
train = x_train
labels = y_train
test = x_test 

####################
### MODEL DESIGN ###
####################
model = boost
# ^^ REPLACE THIS LINE ^^

################
### TRAINING ###
################
# model = model.fit(train, labels)
# ^^ REPLACE THIS LINE ^^

##################
### EVALUATION ###
##################
test_predictions = model_prediction(model)

##################
### SUBMISSION ###
##################
def generate_kaggle_submission(predictions, test = test):
    '''
    This function accepts your 1459-dimensional vector of predicted SalesPrices for the test dataset, 
    and writes a CSV named kaggle_submission.csv containing your vector in a form suitable for 
    submission onto the Kaggle leaderboard.
    '''
    pd.DataFrame({'Id': test_ids, 'SalePrice': predictions}) \
      .to_csv('kaggle_submission.csv', index=False)
    
generate_kaggle_submission(test_predictions)

As you might have noticed in the code block above, we had to write a simple CSV file containing row IDs and predicted values for the 1459 houses in the test dataset. This submission file is your ticket to getting onto the official Kaggle leaderboard and seeing how you did as compared to the rest of the world! 

Take a look at your `kaggle_submission.csv` file to ensure its content matches your expectations. When you and your team are ready, follow these instructions to upload your predictions to Kaggle and receive an official Kaggle score:

> 1. First, choose one person on your team (perhaps the rprincess) to submit your team's predictions under their name. This person will need to visit the [Kaggle website](https://www.kaggle.com) to create an account.
> 2. Go to the [Competition Submission Page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submit) for the House Prices competition. 
> 3. Upload and submit `kaggle_submission.csv`
> 4. Wait for your submission to be scored! This should only take a few seconds to submit. You may have to refresh the page to see your final score.
> 5. To each member: record your team's score in [this Google form](https://goo.gl/forms/VteqJ5Di84t54TEZ2), which also contains a section for feedback on your Kaggle experience this semester. Congratulations on your first Kaggle submission!

For initial comparison, the naive random forest in this notebook achieves a score of 0.15-0.2 - you can think of this as a baseline score to beat.   

This is Patrick's score after some minor edits to the models above:

<img src="GRAPHICS/kaggle.png" width="80%">

Can you beat Patrick's Kaggle score?
> "As an incentive, if you get a score under 0.13, I will personally take your team out to lunch!" - Patrick Chao

If you get some great models and great kaggle scores, feel free to screenshot them and post them in the slack!

We will post a leaderboard of all of the CX Kaggle teams, and the winning team will be honored at banquet, so stay tuned for that! 

In the meantime, take some time to talk to other finished teams and explore the differences in your modeling approach, design, and insights! Happy Kaggling :) 

# Conclusion

This brings us to an end to the CX Kaggle Capstone Project, as well as the Spring 2018 semester of SUSA Career Exploration. Congratulations on graduating from the SUSA Career Exploration committee! It's been a wonderful experience teaching you all, and we hope you got as much out of CX as we did this semester. This semester brought several new pilot programs to CX, such as the crash courses, workbooks, a revamped curriculum, and the CX Kaggle Capstone Project. You all have been great sources of feedback, and we want to make next semester's CX curriculum even better for the new generation of CX! 

We're going to ask you for feedback one last time, to give us insight into how we can improve the CX Kaggle Capstone experience for future CX members. Please fill out [this feedback form](https://goo.gl/forms/VteqJ5Di84t54TEZ2) and let us know how we could have done better. Thank you again for a wonderful semester, and we will see you again in the Fall!

As always, please email [Arun Ramamurthy](mailto:contact@arun.run), [Patrick Chao](mailto:prc@berkeley.edu), or [Noah Gundotra](mailto:noah.gundotra@berkeley.edu) with any questions or concerns whatsoever. Have a great summer, and we hope to see you as a returning member in the Fall! Go SUSA!!!

**Signed with geom_love,  
Lucas, Arun, Patrick, Noah, and the rest of the SUSA Board**